### Import

In [ ]:
%matplotlib widget
import matplotlib
import numpy as np
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import math
import matplotlib.patches as patches
from IPython.display import HTML
import pandas as pd

plt.rcParams['animation.ffmpeg_path'] = '/opt/ffmpeg/bin/ffmpeg'

LVC_path = '/misc/home/u1220/analysis_jupyter'
import sys
sys.path.remove('/home/u1220/lvd_aniso/jupiter_books') if '/home/u1220/lvd_aniso/jupiter_books' in sys.path else None
sys.path.append(LVC_path) if not LVC_path in sys.path else None

from LVC.LVCcase_v2 import LVCcase
plt.style.use('LVC')

### Load data

In [ ]:
%store -r proj_for_trade
proj_for_trade = r'/misc/home/u1220/lvd_aniso/mol_a_LR/results_LR/id_38/Ttarget=53'
print(proj_for_trade)
proj = LVCcase(project_folder=proj_for_trade, auto_load=True, plot=plt)

proj.read_input(verbose=False)

### Tips

In [ ]:
tips_fig = figure(num=None, figsize=(14, 4), dpi=80, facecolor='w')

proj.draw_tipXY(show_stim=True, legend=False)
# plt.xlim(0, 1000)
plt.show()

#### udat

In [ ]:
proj.load_udat(verbose=True, reload=True)
proj.get_frames()
frames = proj.frames

### Draw frame

In [ ]:
frame_fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')
time=6000
proj.draw_frame(plot=plt, N=int(time/proj.dT), show_tips=True, set_title=True, vmax=25, obstacle=True, obstacle_color='magenta', time_shift=1000)

plt.show()

### Animation

In [ ]:
%matplotlib inline
fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')

proj.setup_animation(K=2, first=4000, last=4800, plot=plt, show_tips=True, draw_trace=False, ani_title=True, draw_obstacle=False)
proj.generate_animation(fig)

# proj.animation.save("/misc/home1/u1220/lvc/analysis_jupyter/_trash/SW_formation.gif",  writer='imagemagick', fps=15)
# proj.save_animation(filename='sl14norm_T=232.mp4', overwrite=False, fps=10, folder='/misc/home1/u1220/lvc/analysis_jupyter/_trash/')

%matplotlib widget
HTML(proj.animation.to_jshtml())

### Potential

In [ ]:
x, y = (1,30)

x_ind = int(x/(proj.mult_x*proj.drx))
y_ind = int(y/(proj.mult_y*proj.dry))

proj.load_udat()
time_axis = np.arange(0, proj.frames_number) * proj.dT
        
potential_fig = figure(num=None, figsize=(14, 4), dpi=80, facecolor='w')
plt.plot(list(time_axis), list(proj.get_point_in_time(x_ind,y_ind)), linewidth=1, color='red')
plt.title('Potential at point (%.2f, %.2f), indices (%d, %d)' % 
          (x_ind * proj.drx * proj.mult_x, y_ind * proj.dry * proj.mult_y, x_ind, y_ind))
plt.grid(True)
plt.xlim(900,1200)

plt.show()

### Periods by AP

In [ ]:
x, y = (10,60)
start_t, end_t, u_level = (5000, 6000, -60)
start_t_ind, end_t_ind = int(start_t/proj.dT), int(end_t/proj.dT)

x_ind, y_ind = int(x/(proj.mult_x*proj.drx)), int(y/(proj.mult_y*proj.dry))
times, periods = proj.calculate_periods(x=x_ind, y=y_ind, start=start_t_ind, end=end_t_ind, u_level=u_level)

figure(num=None, figsize=(12, 4), dpi=80, facecolor='w')
# plt.scatter(times, periods, s=7)
plt.plot(times, periods)
plt.grid()
plt.title("Periods at point (%.2f, %.2f) on interval [%.2f, %.2f]. Potential level = %.2f." % 
          (x_ind*proj.drx*proj.mult_x, y_ind*proj.dry*proj.mult_y, start_t_ind*proj.dT, end_t, u_level))
plt.show()
print(np.median(periods))

### APD

In [ ]:
# get APD_X
x, y, start_t, end_t = (90, 30, 0, 1000)

x_ind = int(x/(proj.mult_x*proj.drx))
y_ind = int(y/(proj.mult_y*proj.dry))
start_t_ind = int(start_t/proj.dT)
end_t_ind = int(end_t/proj.dT)

times, apds = proj.calculate_APD_by_percentage(x=x_ind, y=y_ind, start=start_t_ind, end=end_t_ind, APD_percent=.9)

figure(num=None, figsize=(12, 4), dpi=80, facecolor='w')
plt.scatter(times, apds, s=7)
plt.grid()
# plt.title("APD_%d at point (%.2f, %.2f) in interval [%.2f, %.2f]. Potential level = %.2f." % 
#           (APD_level, x_ind*proj.drx*proj.mult_x, y_ind*proj.dry*proj.mult_y, start_t_ind*proj.dT, proj.calc_time, APD_level))
# plt.xlim(0,8000)
# plt.ylim(200,230)
print("Avg APD90: %.2f, Median APD90: %.2f" % (np.average(apds), np.median(apds)))
plt.show()

### Trajectory

In [ ]:
traject_fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')
proj.load_tips()
tips = proj.tips

ax = plt.gca()
ax.set_axisbelow(True)

plt.xlim(xmax = proj.nx * proj.drx, xmin = 0)
plt.ylim(ymax = proj.ny * proj.dry, ymin = 0)
plt.scatter(tips[:,2], tips[:,3], color='red', s=1)
plt.grid(True)

### Wavelength

#### Yline

In [ ]:
time = 100
line_y = 53

FRAME_NUMBER = int(time/proj.dT)
y_level = int(line_y/(proj.mult_y*proj.dry))
print("Y node number %d" % (y_level*int(proj.params['space_mult_y'])))
rest_level = 20 # resting state if lower

figure(num=None, figsize=(12, 4), dpi=80, facecolor='w')
lny = proj.get_Yline(FRAME_NUMBER, y_level)
plt.plot(np.arange(lny.size)*proj.drx*proj.mult_x, lny, marker='.', label='potential')
plt.plot([0, proj.nx*proj.drx],[rest_level, rest_level], label='resting state if lower')
plt.grid()
plt.legend()
plt.show()

frame_fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')
frame = proj.frames[FRAME_NUMBER]
plt.imshow(frame, origin='lower', interpolation='none', extent=[0, proj.nx * proj.drx, 0, proj.ny * proj.dry], vmin=np.amin(proj.frames), vmax=25)
plt.plot([0, proj.nx*proj.drx], [(y_level + 0.5)*proj.dry*proj.mult_y]*2, color='black')
# 0.5 shifting is for better displaying of line -> it crosses middle of squares. 
plt.title('Spiral wave motion at %.2f ms' % (FRAME_NUMBER*proj.dT))
plt.show()

#### Wavelength

In [ ]:
#get wavelength

nxs = proj.nxs
nys = proj.nys
nx = proj.nx
drx = proj.drx
mult_x = proj.mult_x

def get_wavelength(FRAME, y):
    lny = get_Yline(FRAME, y)
    maxs = []
    asc = False
    
    for i in range(1, len(lny) - 1):
        if lny[i] - lny[i - 1] > 1e-10:
            asc = True
        elif lny[i] - lny[i + 1] > 1e-10:
            asc = False
            
        if not asc and lny[i-1] > 2 and lny[i] <= 2 or asc and lny[i-1] <= 2 and lny[i] > 2:
            if not (len(maxs) == 0 and asc):
                maxs.append(i)
        
#         if lny[i] - lny[i - 1] > 0 and lny[i] - lny[i + 1] > 0:
#             maxs.append(i)
    print(maxs)
    wlngth = []
    for i in range(1, len(maxs), 2):
        wlngth.append(maxs[i] - maxs[i-1])
    
    return wlngth

print(np.array(get_wavelength(FRAME_NUMBER, y_level)))